In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from sklearn import linear_model

In [15]:
data = pd.read_csv('data_v1.csv')
name = data.columns.values
name = np.array(['gvkey', 'date', 'fyear', 'fquarter', 'industry_format', 'consol',
       'popsrc', 'datafmt', 'ticker', 'cusip', 'conpany_name', 'currency', 'calendaryr_quarter',
       'fiscalyear_qtr', 'total_asset', 'cash', 'st_debt', 'lt_debt', 'net_income', 'ebitda',
       'int_exp', 'exchange', 'status', 'mkt_val', 'group', 'industry', 'sector',
       'subindustry'])

In [16]:
data.columns = name
data.head()

,gvkey,date,fyear,fquarter,industry_format,consol,popsrc,datafmt,ticker,cusip,...,net_income,ebitda,int_exp,exchange,status,mkt_val,group,industry,sector,subindustry
0,1004,19900228,1989,3,INDL,C,D,STD,AIR,361105,...,6.109,14.251,2.758,11,A,NaN,2010.0,201010.0,20.0,20101010.0
1,1004,19900531,1989,4,INDL,C,D,STD,AIR,361105,...,6.224,13.137,2.309,11,A,NaN,2010.0,201010.0,20.0,20101010.0
2,1004,19900831,1990,1,INDL,C,D,STD,AIR,361105,...,6.697,15.400,2.607,11,A,NaN,2010.0,201010.0,20.0,20101010.0
3,1004,19901130,1990,2,INDL,C,D,STD,AIR,361105,...,0.126,8.871,2.708,11,A,NaN,2010.0,201010.0,20.0,20101010.0
4,1004,19910228,1990,3,INDL,C,D,STD,AIR,361105,...,3.977,11.344,2.587,11,A,NaN,2010.0,201010.0,20.0,20101010.0


In [112]:
# there is no dual-listing
a = data[['ticker','exchange']].drop_duplicates().groupby('ticker').count()
a[a['exchange'] != 1]

,exchange
ticker,


In [17]:
# how many company compustst is inactively updating the data
# when compustat stop track the data, we believe the company will not be included in the top 1000 market cap range.
# so not affecting our analysis
len(data[data['status']== 'I']['ticker'].unique())

8536

In [18]:
len(data[data['status']== 'A']['ticker'].unique())

5576

In [61]:
# check if any company have 2 status
# if a company is not updated by the database anymore,
# the status changed to I for each time period before

inactive = data[data['status']== 'I']['ticker'].unique()
active = data[data['status']== 'A']['ticker'].unique()
duplicates = []
for ticker in inactive:
    if ticker in active:
        duplicates.append(inactive[i])
duplicates


[]

In [60]:
# 0(n)
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 
   
print(intersection(inactive, active)) 

[]


In [53]:
len(data['ticker'].unique())

14111

In [115]:
data[data['ticker']=='TAM']

,gvkey,date,fyear,fquarter,industry_format,consol,popsrc,datafmt,ticker,cusip,...,net_income,ebitda,int_exp,exchange,status,mkt_val,group,industry,sector,subindustry
666554,311524,20100331,2010,1,INDL,C,D,STD,TAM,87509U106,...,NaN,NaN,NaN,11,I,NaN,1510.0,151010.0,15.0,15101050.0
666555,311524,20100630,2010,2,INDL,C,D,STD,TAM,87509U106,...,NaN,NaN,NaN,11,I,NaN,1510.0,151010.0,15.0,15101050.0
666556,311524,20100930,2010,3,INDL,C,D,STD,TAM,87509U106,...,NaN,NaN,NaN,11,I,NaN,1510.0,151010.0,15.0,15101050.0
666557,311524,20101231,2010,4,INDL,C,D,STD,TAM,87509U106,...,NaN,NaN,NaN,11,I,NaN,1510.0,151010.0,15.0,15101050.0
666558,311524,20110331,2011,1,INDL,C,D,STD,TAM,87509U106,...,NaN,NaN,NaN,11,I,NaN,1510.0,151010.0,15.0,15101050.0
666559,311524,20110630,2011,2,INDL,C,D,STD,TAM,87509U106,...,NaN,NaN,NaN,11,I,NaN,1510.0,151010.0,15.0,15101050.0
666560,311524,20110930,2011,3,INDL,C,D,STD,TAM,87509U106,...,NaN,NaN,NaN,11,I,NaN,1510.0,151010.0,15.0,15101050.0
666561,311524,20111231,2011,4,INDL,C,D,STD,TAM,87509U106,...,30.0,226.9,75.4,11,I,NaN,1510.0,151010.0,15.0,15101050.0
666562,311524,20120331,2012,1,INDL,C,D,STD,TAM,87509U106,...,-43.0,9.0,11.0,11,I,NaN,1510.0,151010.0,15.0,15101050.0
666563,311524,20120630,2012,2,INDL,C,D,STD,TAM,87509U106,...,7.0,68.0,19.0,11,I,NaN,1510.0,151010.0,15.0,15101050.0


In [129]:
# ret is the table we will be used for analysis
ret = data[['date', 'ticker', 'conpany_name','calendaryr_quarter', 'total_asset', 'st_debt', 'lt_debt', 'net_income', 'ebitda',
       'int_exp', 'exchange', 'status', 'mkt_val','industry', 'sector']].copy()
ret['ROA'] = ret['net_income']/ret['total_asset']
ret['total_debt'] = ret['st_debt']+ret['lt_debt']
ret['D/A'] = ret['total_debt']+ret['total_asset']

In [131]:
ret.head()

,date,ticker,conpany_name,calendaryr_quarter,total_asset,st_debt,lt_debt,net_income,ebitda,int_exp,exchange,status,mkt_val,industry,sector,ROA,total_debt,D/A
0,19900228,AIR,AAR CORP,1990Q1,383.790,32.551,72.606,6.109,14.251,2.758,11,A,NaN,201010.0,20.0,0.015918,105.157,488.947
1,19900531,AIR,AAR CORP,1990Q2,388.521,33.821,72.329,6.224,13.137,2.309,11,A,NaN,201010.0,20.0,0.016020,106.150,494.671
2,19900831,AIR,AAR CORP,1990Q3,385.752,27.427,71.806,6.697,15.400,2.607,11,A,NaN,201010.0,20.0,0.017361,99.233,484.985
3,19901130,AIR,AAR CORP,1990Q4,382.978,33.563,71.769,0.126,8.871,2.708,11,A,NaN,201010.0,20.0,0.000329,105.332,488.310
4,19910228,AIR,AAR CORP,1991Q1,383.354,11.436,69.020,3.977,11.344,2.587,11,A,NaN,201010.0,20.0,0.010374,80.456,463.810
